In [78]:
import requests
import pandas as pd
import os 

In [79]:
BASE_URL = 'https://fantasy.premierleague.com/api/'
GENERAL = 'bootstrap-static/'  

In [80]:
type_mapping = {
        1: "GK",
        2: "DEF",
        3: "MID",
        4: "FWD",
        'u': "Unavailable",
        'a': "Available",
        'd': "Doubtful",
        'NaN': "Info unavailable"
    }

In [81]:
def fetch_fpl_data():
    """Fetch the latest FPL data and return it as a pandas DataFrame."""
    url = BASE_URL + GENERAL
    r = requests.get(url).json()
    players_df = pd.DataFrame(r['elements'])
    return players_df

In [82]:
df = fetch_fpl_data()

In [83]:
df.columns

Index(['can_transact', 'can_select', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'code', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points',
       'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added',
       'now_cost', 'photo', 'points_per_game', 'removed', 'second_name',
       'selected_by_percent', 'special', 'squad_number', 'status', 'team',
       'team_code', 'total_points', 'transfers_in', 'transfers_in_event',
       'transfers_out', 'transfers_out_event', 'value_form', 'value_season',
       'web_name', 'region', 'team_join_date', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 

In [84]:
def load_yesterday_costs():
    """
    Load yesterday's cost data from GitHub private repository.
    Return an empty DataFrame if the file cannot be accessed.
    """
    # Use raw content URL
    github_url = "https://raw.githubusercontent.com/meharpalbasi/fpl_price_change_daily/main/yesterday_costs.csv"
    
    try:
        response = requests.get(github_url, verify=False)
        response.raise_for_status()
        df = pd.read_csv(pd.io.common.StringIO(response.text))
        print(f"Loaded {len(df)} records from yesterday's data")
        print("Yesterday's data columns:", df.columns.tolist())
        return df
    except Exception as e:
        print(f"Error loading yesterday's costs: {e}")
        return pd.DataFrame()

In [85]:
def format_price_changes(df, change_type):
        arrow = '🔽' if change_type == 'Price Falls' else '🔼'
        message = f"{change_type} {arrow}\n\n"
        message += "Player New Price\n" # Previous
        
        for _, player in df.iterrows():
            message += f"{player['web_name']}  £{player['now_cost']:.1f}\n" # £{player['prev_cost']:.1f}
        return message.strip()  

In [86]:
def format_price_changes(df, change_type):
    """
    Return a formatted string listing the players who have
    changed price (up or down).
    """
    arrow = '🔽' if change_type == 'Price Falls' else '🔼'
    message = f"{change_type} {arrow}\n\n"
    message += "Player   New Price   Old Price\n"
    
    for _, player in df.iterrows():
        message += (f"{player['web_name']}   "
                    f"£{player['now_cost']/10:.1f}   "
                    f"£{player['prev_cost']/10:.1f}\n")
    return message.strip()

In [87]:
def main():
    # 1. Fetch today's data
    players_df = fetch_fpl_data()
    
    # 2. Load yesterday's data
    yesterday_df = load_yesterday_costs()
    # 3. Clean up today's data with your typical approach:
    players_clean_df = players_df[["id", "web_name", "now_cost", "cost_change_event"]].assign(
        position=players_df['element_type'].map(type_mapping),
        availability=players_df['status'].map(type_mapping)
    )
    if yesterday_df.empty:
        print("Warning: No yesterday data loaded!")
        return

    # 4. Merge with yesterday's data
    merged_df = players_clean_df.merge(
        yesterday_df, 
        on="id", 
        how="left", 
        suffixes=("", "_yest")
    )

    # 5. Compute the daily change by comparing now_cost to yesterday's now_cost
    merged_df["daily_change"] = merged_df["now_cost"] - merged_df["now_cost_yest"]

    # 6. Create columns for new/previous cost for display
    merged_df["prev_cost"] = merged_df["now_cost_yest"]
    
    # 7. Filter out players who changed price
    price_changed_players = merged_df[merged_df["daily_change"] != 0].copy()
    
    # 8. Add an arrow column (up/down) based on daily_change
    price_changed_players["arrow"] = price_changed_players["daily_change"].apply(
        lambda x: "up" if x > 0 else ("down" if x < 0 else "no_change")
    )

    # 9. Separate rises and falls
    price_rises = price_changed_players[price_changed_players["arrow"] == "up"]
    price_falls = price_changed_players[price_changed_players["arrow"] == "down"]

    # 10. Sort them (optional: e.g., by now_cost ascending)
    price_rises = price_rises.sort_values(by='now_cost', ascending=True)
    price_falls = price_falls.sort_values(by='now_cost', ascending=True)

    # 11. Format messages
    falls_message = format_price_changes(price_falls, 'Price Falls')
    rises_message = format_price_changes(price_rises, 'Price Rises')

    # 12. Print or send the messages
    print(falls_message)
    print("\n" + "-"*40 + "\n")
    print(rises_message)

In [88]:
if __name__ == "__main__":
    main()

/Users/meharpalbasi/Documents/fpl telegram bot/venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Loaded 723 records from yesterday's data
Yesterday's data columns: ['id', 'now_cost']
Price Falls 🔽

Player   New Price   Old Price
Heaton   £3.9   £4.0
Kelleher   £4.1   £4.2
Pau   £4.2   £4.3
O'Brien   £4.5   £4.6
Gomez   £4.7   £4.8
Casemiro   £4.7   £4.8
Pedro Porro   £5.4   £5.5
Strand Larsen   £5.4   £5.5
Madueke   £6.1   £6.2
Kulusevski   £6.4   £6.5
G.Jesus   £6.8   £6.9
Bowen   £7.3   £7.4
Luis Díaz   £7.5   £7.6

----------------------------------------

Price Rises 🔼

Player   New Price   Old Price
Muñoz   £4.8   £4.7
Aina   £5.3   £5.2
Wood   £6.9   £6.8
Mateta   £7.3   £7.2
B.Fernandes   £8.4   £8.3


,web_name,cost_change_event,now_cost,position,availability
0,Fábio Vieira,0,54,MID,Unavailable
1,G.Jesus,0,69,FWD,Available
2,Gabriel,0,64,DEF,Available
3,Havertz,-1,78,FWD,Available
4,Hein,0,40,GK,Unavailable
...,...,...,...,...,...
713,André,0,50,MID,Doubtful
714,Forbs,0,55,MID,Available
715,Pond,0,39,DEF,Available
716,Edozie,0,45,MID,Available
